# pywinauto
pywinauto 适合哪些中规中矩的 有各种菜单的 传统程序
比较方便操作菜单
+ 应用控制（打开、关闭应用等）
+ 窗口控制（最大化、最小化窗口）
+ 控件操作 如编辑

In [ ]:
import time

In [ ]:
from pywinauto.application import Application

app = Application(backend='win32')

In [ ]:
# 打开程序
app.start(r'C:\Windows\regedit.exe') 
# 必须是有效的 Win32 应用程序 且 必须是绝对路径

In [ ]:
# 连接程序 
app.connect(path=r'C:\Windows\regedit.exe')
# 待连接的应用必须是已经打开的应用

In [ ]:
dlg = app.window()
# 程序可查的几个属性
a1 = app.backend
a2 = app.enabled_only
a3 = app.visible_only
a4 = app.process
a5 = app.SysTreeView32
print(a1)
print(a2)
print(a3)
print(a4)
print(a5)

In [ ]:
# 获取窗口信息
# app.RegEdit_RegEdit.print_control_identifiers() # 调用程序菜单信息
# 得出结论：pywinauto 只适合哪些中规中矩的 有各种菜单的 传统程序

In [ ]:
# 关闭程序
app.kill()

# pywin32

### 获得主窗口句柄

In [ ]:
import win32gui, win32con

Handle = win32gui.FindWindow('TdxW_MainFrame_Class', None) # 通达信模拟交易终端V8.09 - [行情报价-自选股]
Handle

### 主窗 显示方式的设置

In [ ]:
win32gui.ShowWindow(Handle,win32con.SW_RESTORE)

### 最大化 最小化

In [ ]:
win32gui.PostMessage(Handle, win32con.WM_SYSCOMMAND, win32con.SC_MAXIMIZE, 0)

In [ ]:
win32gui.PostMessage(Handle, win32con.WM_SYSCOMMAND, win32con.SC_MINIMIZE, 0)

### 判断窗口是否最小化

In [ ]:
if win32gui.IsIconic(Handle):
    print('最小化了')
    win32gui.ShowWindow(Handle,win32con.SW_RESTORE)
    print('已恢复')
else:
    print('没最小化')

### 关闭窗口

In [ ]:
win32gui.PostMessage(Handle, win32con.WM_CLOSE, 0, 0)

### 前置窗口

In [ ]:
win32gui.SetForegroundWindow(Handle)

### 聚焦窗口

In [ ]:
try:
    win32gui.SetFocus(Handle)
except Exception as e:
    print('错误：', e)
else:
    pass

### 获取窗边界

In [ ]:
x, y, x_, y_ = win32gui.GetWindowRect(Handle)
print('起始坐标：(%d: %d)\n右下坐标：(%d: %d)'\
      %(x, y, x_, y_))

### 判断屏幕是否缩放
**使用pyautogui时, 缩放比必须是1.0**

In [ ]:
import win32api, win32con, win32print, win32gui

def get_screen_scale_rate():
    x = win32api.GetSystemMetrics(0)  # 屏幕缩放后的宽度分辨率
    hDC = win32gui.GetDC(0)
    w = win32print.GetDeviceCaps(hDC, win32con.DESKTOPHORZRES)  # 屏幕真实的宽度分辨率
    rate = w / x
    return rate
r = get_screen_scale_rate()
print('屏幕缩放比：', r)

## 移动窗口+改变窗口大小

In [ ]:
win32gui.SetWindowPos(
    Handle, 
    win32con.HWND_TOPMOST, 
    0, 0, 700, 700, 
    win32con.SWP_SHOWWINDOW
)

### 鼠标后台点击

In [ ]:
def doClick(hwnd,cx,cy):
    '''
    hwnd：要点击的窗口句柄 必须是本身句柄 不能是父句柄
    cx cy：点击位置位于该窗口的相对位置
    '''
        long_position = win32api.MAKELONG(cx, cy)#模拟鼠标指针 传送到指定坐标
        win32api.SendMessage(hwnd, win32con.WM_LBUTTONDOWN, win32con.MK_LBUTTON, long_position)#模拟鼠标按下
        win32api.SendMessage(hwnd, win32con.WM_LBUTTONUP, win32con.MK_LBUTTON, long_position)#模拟鼠标弹起

### 已知主窗口句柄，循环递归找孙窗口句柄

In [ ]:
import win32gui, win32con, win32api


def find_brother_Handle(pHandle, winClass, index=0):
    """
    已知子窗口的窗体类名
    寻找第index号个同类型的兄弟窗口
    """
    assert type(index) == int and index >= 0
    handle = win32gui.FindWindowEx(pHandle, 0, winClass, None)
    while index > 0:
        handle = win32gui.FindWindowEx(pHandle, handle, winClass, None)
        index -= 1
    return handle


def find_sub_Handle(pHandle, winClassList):
    """
    递归寻找子窗口的句柄
    pHandle是祖父窗口的句柄
    winClassList是各个子窗口的class列表，父辈的list-index小于子辈
    """
    assert type(winClassList) == list
    if len(winClassList) == 1:
        return find_brother_Handle(pHandle, winClassList[0][0], winClassList[0][1])
    else:
        pHandle = find_brother_Handle(pHandle, winClassList[0][0], winClassList[0][1])
        return find_sub_Handle(pHandle, winClassList[1:])



pHandle = win32gui.FindWindow('RegEdit_RegEdit','注册表编辑器')
son_handle = win32gui.FindWindowEx(pHandle, 0, 'SysTreeView32', None)


index_son_handle = find_brother_Handle(pHandle,'SysTreeView32',1)
grandson_handle= find_sub_Handle(pHandle,[("SysListView32", 0), ("SysHeader32", 0)])
#            主父窗口 [(子窗口类型,子窗口序号),(孙窗口类型,孙窗口序号)]

print('主父窗口：',pHandle)
print('直接子窗口：',son_handle)
print('2号子窗口：',index_son_handle)
print('孙窗口：',grandson_handle)

# pyautogui
+ 屏幕控制（获取屏幕尺寸、截屏）
+ 鼠标控制（移动鼠标、单击、双击、右击、拖拽、滚动）
+ 键盘控制（编辑、按键）

In [ ]:
import pyautogui

### 获取屏幕分辨率

In [ ]:
screen_Width, screen_Height = pyautogui.size()

'屏幕分辨率：%d * %d'%(screen_Width, screen_Height)

### 截图

In [ ]:
# 部分截屏
pyautogui.screenshot('test_file/部分截屏.png', region = (0,0,60,60))
# 截取整屏
img = pyautogui.screenshot()
img.save('test_file/全屏截屏.png')

### 找图

In [ ]:
# 在当前屏幕中查找指定图片(图片是当前屏幕某个区域的截图)
# 返回anchor
anchor = pyautogui.locateOnScreen(
    'test_file/部分截屏.png', 
    region=(0, 0, 1920, 1080),
#    confidence=0.9,  # 精度 需要安装openCV包
    grayscale=True  # 灰度 可提升找图速度
) 

if anchor:
    x, y = pyautogui.center(anchor) # 获取定位到的图中间点坐标
else:
    x, y = -1, -1
    print('找不到')

'找到图的中心坐标：', x, y

In [ ]:
try:
    x, y = pyautogui.locateCenterOnScreen('test_file/右下角网络图标.png', region=(0, 0, 1920, 1080)) # 返回x,y
except Exception as e:
    print(e)
else:
    print('找到图的中心坐标：', x, y)

### 鼠标移动

In [ ]:
x, y = pyautogui.position()
print('鼠标即时位置：', x, y)

# 移动鼠标（x, y 为绝对位置，xOffset, yOffset 为相对偏移量，tween：渐变函数）
pyautogui.moveTo(x=100, y=100, duration=2, tween=pyautogui.linear)
pyautogui.moveRel(xOffset=500, yOffset=500, duration=2, tween=pyautogui.linear)

### 鼠标点击

### 鼠标拖拽

### 鼠标滚动

### 输入文本

In [ ]:
# esc + l 可以调出这种


### 按键

In [ ]:
# 单按
pyautogui.press('esc')

#依次按 多个按键
pyautogui.press(['left', 'left', 'left'])
 
# 组合热键hotkey（从左到右依次按下按键，再从右到左依次松开按键）
pyautogui.hotkey('ctrl', 'c')

### 消息窗口